In [ ]:
import math
import numpy as np
import torch
import torchvision.transforms as T
# from decord import VideoReader, cpu  # Commented out since not needed for basic usage
from PIL import Image
from torchvision.transforms.functional import InterpolationMode

import torch
from transformers import AutoConfig, AutoTokenizer, AutoModel

print("🔧 Loading InternVL3-2B model...")
model_path = "/home/jovyan/nfs_share/models/InternVL3-2B"
model = AutoModel.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    use_flash_attn=False,
    trust_remote_code=True).eval().cuda()

print("✅ Model loaded successfully")

In [2]:
# from transformers_modules.OpenGVLab.InternVL3.tokenization_internvl import InternVLTokenizer
# tokenizer = InternVLTokenizer.from_pretrained(model_path)

# # Look for the tokenizer class in the model directory
# import os
# import sys

# # Add the model directory to Python path
# model_path = "/home/jovyan/nfs_share/models/huggingface/hub/InternVL3-1B"
# sys.path.append(model_path)

# # Verify what's in the directory
# print(os.listdir(model_path))

# # Then try importing the tokenizer from there
# from tokenization_internvl import InternVLTokenizer
# tokenizer = InternVLTokenizer.from_pretrained(model_path)

In [ ]:
# Fixed tokenizer loading for InternVL3-2B
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_path, 
    trust_remote_code=True, 
    use_fast=False  # Important for InternVL3
)
print("✅ Tokenizer loaded successfully")

In [ ]:
model

## [Quick Start](https://huggingface.co/OpenGVLab/InternVL3-1B#quick-start)

In [ ]:
# Simple image processing (from official InternVL3 docs)
def load_image(image, input_size=448):
    """Simple image preprocessing following official InternVL3 docs"""
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size)),
        T.ToTensor(),
        T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
    ])
    return transform(image).unsqueeze(0).to(torch.bfloat16).cuda()

# Load and process image
imageName = "/home/jovyan/nfs_share/tod/datasets/synthetic_invoice_014.png"
image = Image.open(imageName)
print(f"📷 Image loaded: {image.size}")

print("🖼️  Processing image...")
pixel_values = load_image(image)
print(f"✅ Image processed: {pixel_values.shape}")

# Generation config
generation_config = dict(max_new_tokens=1024, do_sample=True)

# Test simple image conversation
question = '<image>\nPlease describe the image shortly.'
print(f"❓ Question: {question}")

print("🤖 Generating response...")
try:
    response = model.chat(tokenizer, pixel_values, question, generation_config)
    print("✅ Response generated successfully!")
    print("\n" + "="*50)
    print("RESPONSE:")
    print(response)
    print("="*50)
    
except Exception as e:
    print(f"❌ Error during inference: {e}")
    print(f"Error type: {type(e).__name__}")
    import traceback
    traceback.print_exc()

In [ ]:
# Save response to file (optional)
from pathlib import Path

try:
    output_path = Path("/home/jovyan/nfs_share/tod/output/internvl3_ibm_output.txt")
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    with output_path.open("w", encoding="utf-8") as text_file:
        text_file.write(response)
    
    print(f"✅ Response saved to: {output_path}")
    print(f"📄 File size: {output_path.stat().st_size} bytes")
    
except NameError:
    print("❌ Error: 'response' variable not defined.")
    print("💡 Please run the previous cell first to generate the response.")
    
except Exception as e:
    print(f"❌ Error saving file: {e}")
    print(f"💡 Check if directory exists: {output_path.parent}")